# 11.6 Resampling and Frequency Conversion（重采样和频度转换）

重采样（Resampling）指的是把时间序列的频度变为另一个频度的过程。把高频度的数据变为低频度叫做降采样（downsampling），把低频度变为高频度叫做增采样（upsampling）。并不是所有的重采样都会落入上面这几个类型，例如，把W-WED（weekly on Wednesday）变为W-FRI，既不属于降采样，也不属于增采样。
pandas对象自带resampe方法，用于所有的频度变化。resample有一个和groupby类似的API；我们可以用resample来对数据进行分组，然后调用聚合函数（aggregation function）:

In [1]:
import numpy as np
import pandas as pd

In [2]:
rng = pd.date_range('2000-01-01', periods=100, freq='D')

In [3]:
ts = pd.Series(np.random.randn(len(rng)), index=rng)
ts

2000-01-01    0.125437
2000-01-02   -0.928762
2000-01-03   -0.683013
2000-01-04   -0.096907
2000-01-05    1.393276
2000-01-06   -0.175641
2000-01-07    0.636677
2000-01-08   -0.248456
2000-01-09    0.358301
2000-01-10   -1.851011
2000-01-11    0.730459
2000-01-12    0.384839
2000-01-13   -2.791967
2000-01-14   -0.638036
2000-01-15   -0.652604
2000-01-16    0.041689
2000-01-17    1.457772
2000-01-18   -1.381615
2000-01-19    0.620395
2000-01-20   -1.381277
2000-01-21    1.200036
2000-01-22    1.080177
2000-01-23   -0.597535
2000-01-24    2.281261
2000-01-25    0.327792
2000-01-26    1.466569
2000-01-27    0.482982
2000-01-28   -0.415015
2000-01-29    1.004746
2000-01-30    0.350312
                ...   
2000-03-11   -0.241458
2000-03-12    0.675380
2000-03-13   -0.548206
2000-03-14   -1.409016
2000-03-15   -0.642607
2000-03-16    1.259163
2000-03-17   -0.096923
2000-03-18   -0.296001
2000-03-19   -0.045976
2000-03-20    0.089562
2000-03-21   -0.317393
2000-03-22   -0.298328
2000-03-23 

In [4]:
ts.resample('M').mean()

2000-01-31    0.113179
2000-02-29   -0.233165
2000-03-31    0.152156
2000-04-30   -0.250607
Freq: M, dtype: float64

In [5]:
ts.resample('M', kind='period').mean()

2000-01    0.113179
2000-02   -0.233165
2000-03    0.152156
2000-04   -0.250607
Freq: M, dtype: float64

resample是一个灵活且高效的方法，可以用于处理大量的时间序列。下面是一些相关的选项：

![](http://oydgk2hgw.bkt.clouddn.com/pydata-book/nwc0s.png)

# 1 Downsampling（降采样）

把数据聚合为规律、低频度是一个很普通的时间序列任务。用于处理的数据不必是有固定频度的；我们想要设定的频度会定义箱界（bin edges），根据bin edges会把时间序列分割为多个片段，然后进行聚合。例如，转换为月度，比如'M'或'BM'，我们需要把数据以月为间隔进行切割。每一个间隔都是半开放的（half-open）；一个数据点只能属于一个间隔，所有间隔的合集，构成整个时间范围（time frame）。当使用resample去降采样数据的时候，有很多事情需要考虑：

- 在每个间隔里，哪一边要闭合
- 怎样对每一个聚合的bin贴标签，可以使用间隔的开始或结束

为了演示一下，下面用一个一分钟的数据来举例：

In [6]:
rng = pd.date_range('2000-01-01', periods=12, freq='T')

In [7]:
ts = pd.Series(np.arange(12), index=rng)
ts

2000-01-01 00:00:00     0
2000-01-01 00:01:00     1
2000-01-01 00:02:00     2
2000-01-01 00:03:00     3
2000-01-01 00:04:00     4
2000-01-01 00:05:00     5
2000-01-01 00:06:00     6
2000-01-01 00:07:00     7
2000-01-01 00:08:00     8
2000-01-01 00:09:00     9
2000-01-01 00:10:00    10
2000-01-01 00:11:00    11
Freq: T, dtype: int32

假设我们想要按5分钟一个数据块来进行聚合，然后对每一个组计算总和：

In [8]:
ts.resample('5min', closed='right').sum()

1999-12-31 23:55:00     0
2000-01-01 00:00:00    15
2000-01-01 00:05:00    40
2000-01-01 00:10:00    11
Freq: 5T, dtype: int32

我们传入的频度定义了每个bin的边界按5分钟递增。默认，bin的左边界是闭合的，所以`00:00`值是属于`00:00`到`00:05`间隔的。设定closed='right'，会让间隔的右边闭合：

In [9]:
ts.resample('5min', closed='right').sum()

1999-12-31 23:55:00     0
2000-01-01 00:00:00    15
2000-01-01 00:05:00    40
2000-01-01 00:10:00    11
Freq: 5T, dtype: int32

默认，每一个bin的左边的时间戳，会被用来作为结果里时间序列的标签。通过设置label='right'，我们可以使用bin右边的时间戳来作为标签：

In [10]:
ts.resample('5min', closed='right', label='right').sum()

2000-01-01 00:00:00     0
2000-01-01 00:05:00    15
2000-01-01 00:10:00    40
2000-01-01 00:15:00    11
Freq: 5T, dtype: int32

可以看下图方便理解：

![](http://oydgk2hgw.bkt.clouddn.com/pydata-book/d770h.png)

最后，我们可能想要对结果的索引进行位移，比如在右边界减少一秒。想要实现的话，传递一个字符串或日期偏移给loffset：

In [11]:
ts.resample('5min', closed='right', 
            label='right', loffset='-1s').sum()

1999-12-31 23:59:59     0
2000-01-01 00:04:59    15
2000-01-01 00:09:59    40
2000-01-01 00:14:59    11
Freq: 5T, dtype: int32

我们也可以使用shift方法来实现上面loffset的效果。

### Open-High-Low-Close (OHLC) resampling（股价图重取样）

>  Open-High-Low-Close: 开盘-盘高-盘低-收盘图；股票图；股价图

在经济界，一个比较流行的用法，是对时间序列进行聚合，计算每一个桶（bucket）里的四个值：first（open），last（close），maximum（high），minimal（low），即开盘-收盘-盘高-盘低，四个值。使用ohlc聚合函数可以得到这四个聚合结果：

In [12]:
ts.resample('5min').ohlc()

,open,high,low,close
2000-01-01 00:00:00,0,4,0,4
2000-01-01 00:05:00,5,9,5,9
2000-01-01 00:10:00,10,11,10,11


# 2 Upsampling and Interpolation（增采样和插值）

把一个低频度转换为高频度，是不需要进行聚合的。下面是一个有周数据的DataFrame：

In [13]:
frame = pd.DataFrame(np.random.randn(2, 4),
                     index=pd.date_range('1/1/2000', periods=2,
                                         freq='W-WED'),
                     columns=['Colorado', 'Texas', 'New York', 'Ohio'])
frame

,Colorado,Texas,New York,Ohio
2000-01-05,-0.706579,-0.572741,0.737777,0.993188
2000-01-12,-1.314004,-0.790550,0.437713,-1.433978


当我们对这个数据进行聚合的的时候，每个组只有一个值，以及gap（间隔）之间的缺失值。在不使用任何聚合函数的情况下，我们使用asfreq方法将其转换为高频度：

In [14]:
df_daily = frame.resample('D').asfreq()
df_daily

,Colorado,Texas,New York,Ohio
2000-01-05,-0.706579,-0.572741,0.737777,0.993188
2000-01-06,NaN,NaN,NaN,NaN
2000-01-07,NaN,NaN,NaN,NaN
2000-01-08,NaN,NaN,NaN,NaN
2000-01-09,NaN,NaN,NaN,NaN
2000-01-10,NaN,NaN,NaN,NaN
2000-01-11,NaN,NaN,NaN,NaN
2000-01-12,-1.314004,-0.790550,0.437713,-1.433978


假设我们想要用每周的值来填写非周三的部分。这种方法叫做填充（filling）或插值（interpolation），可以使用fillna或reindex方法来实现重采样：

In [15]:
frame.resample('D').ffill()

,Colorado,Texas,New York,Ohio
2000-01-05,-0.706579,-0.572741,0.737777,0.993188
2000-01-06,-0.706579,-0.572741,0.737777,0.993188
2000-01-07,-0.706579,-0.572741,0.737777,0.993188
2000-01-08,-0.706579,-0.572741,0.737777,0.993188
2000-01-09,-0.706579,-0.572741,0.737777,0.993188
2000-01-10,-0.706579,-0.572741,0.737777,0.993188
2000-01-11,-0.706579,-0.572741,0.737777,0.993188
2000-01-12,-1.314004,-0.790550,0.437713,-1.433978


我们可以选择只对一部分的周期进行填写：

In [16]:
frame.resample('D').ffill(limit=2)

,Colorado,Texas,New York,Ohio
2000-01-05,-0.706579,-0.572741,0.737777,0.993188
2000-01-06,-0.706579,-0.572741,0.737777,0.993188
2000-01-07,-0.706579,-0.572741,0.737777,0.993188
2000-01-08,NaN,NaN,NaN,NaN
2000-01-09,NaN,NaN,NaN,NaN
2000-01-10,NaN,NaN,NaN,NaN
2000-01-11,NaN,NaN,NaN,NaN
2000-01-12,-1.314004,-0.790550,0.437713,-1.433978


注意，新的日期索引不能与旧的有重叠：

In [17]:
frame.resample('W-THU').ffill()

,Colorado,Texas,New York,Ohio
2000-01-06,-0.706579,-0.572741,0.737777,0.993188
2000-01-13,-1.314004,-0.790550,0.437713,-1.433978


# 3 Resampling with Periods（对周期进行重采样）

对周期的索引进行重采样的过程，与之前时间戳的方法相似：

In [18]:
frame = pd.DataFrame(np.random.randn(24, 4),
                     index=pd.period_range('1-2000', '12-2001',
                                           freq='M'),
                     columns=['Colorado', 'Texas', 'New York', 'Ohio'])
frame[:5]

,Colorado,Texas,New York,Ohio
2000-01,-0.664084,-0.256652,-0.789992,-0.830034
2000-02,-0.275150,-0.132876,0.506776,-0.604915
2000-03,-1.712982,-0.348685,1.206385,0.266092
2000-04,-2.973719,0.369962,0.010205,-0.713987
2000-05,-0.134460,-0.736039,0.332593,-0.971514


In [19]:
annual_frame = frame.resample('A-DEC').mean()
annual_frame

,Colorado,Texas,New York,Ohio
2000,-0.430382,0.124628,0.292538,-0.520008
2001,-0.246860,0.098077,-0.112346,0.186938


增采样需要考虑的要多一些，比如在重采样前，选择哪一个时间跨度作为结束，就像asfreq方法那样。convertion参数默认是'start'，但也能用'end'：

In [20]:
# Q-DEC: Quarterly, year ending in December
annual_frame.resample('Q-DEC').ffill()

,Colorado,Texas,New York,Ohio
2000Q1,-0.430382,0.124628,0.292538,-0.520008
2000Q2,-0.430382,0.124628,0.292538,-0.520008
2000Q3,-0.430382,0.124628,0.292538,-0.520008
2000Q4,-0.430382,0.124628,0.292538,-0.520008
2001Q1,-0.246860,0.098077,-0.112346,0.186938
2001Q2,-0.246860,0.098077,-0.112346,0.186938
2001Q3,-0.246860,0.098077,-0.112346,0.186938
2001Q4,-0.246860,0.098077,-0.112346,0.186938


In [21]:
annual_frame.resample('Q-DEC', convention='end').ffill()

,Colorado,Texas,New York,Ohio
2000Q4,-0.430382,0.124628,0.292538,-0.520008
2001Q1,-0.430382,0.124628,0.292538,-0.520008
2001Q2,-0.430382,0.124628,0.292538,-0.520008
2001Q3,-0.430382,0.124628,0.292538,-0.520008
2001Q4,-0.246860,0.098077,-0.112346,0.186938


增采样和降采样的规则更严格一些：

- 降采样中，目标频度必须是原频度的子周期（subperiod）
- 增采样中，目标频度必须是原频度的母周期（superperiod）

如果不满足上面的规则，会报错。主要会影响到季度，年度，周度频度；例如，用Q-MAR定义的时间跨度只与A-MAR, A-JUN, A-SEP, A-DEC进行对齐（line up with）：

In [22]:
annual_frame.resample('Q-MAR').ffill()

,Colorado,Texas,New York,Ohio
2000Q4,-0.430382,0.124628,0.292538,-0.520008
2001Q1,-0.430382,0.124628,0.292538,-0.520008
2001Q2,-0.430382,0.124628,0.292538,-0.520008
2001Q3,-0.430382,0.124628,0.292538,-0.520008
2001Q4,-0.246860,0.098077,-0.112346,0.186938
2002Q1,-0.246860,0.098077,-0.112346,0.186938
2002Q2,-0.246860,0.098077,-0.112346,0.186938
2002Q3,-0.246860,0.098077,-0.112346,0.186938
